In [18]:
import sys

sys.path.append("../")
from pathlib import Path
import polars as pl
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

from common.make_vec_env import make_vec_env
from envs.single_stock_trading_reward_only_at_sell import StockTradingEnv

In [13]:
%reload_ext autoreload
%autoreload 2

In [14]:
TICKER = "SBIN.NS"
TRAIN_FILE = Path("../datasets") / f"{TICKER}_train"
EVAL_FILE = Path("datasets") / f"{TICKER}_trade"

In [15]:
import numpy as np
import polars as pl
from pathlib import Path

def load_close_prices(TICKER):
    TRAIN_FILE = Path("../datasets") / f"{TICKER}"

    CLOSE_PRICES = np.array(
        pl
        .read_parquet(TRAIN_FILE)
        .with_columns(index=pl.int_range(0, end=pl.count(), eager=False))
        .sort("index")
        .set_sorted("index")
        .group_by_dynamic(
            "index", every="1i", period="40i", include_boundaries=True, closed="right"
        )
        .agg(pl.col("Close"))
        .with_columns(pl.col("Close").list.len().alias("Count"))
        .filter(pl.col("Count") == 40)
        ["Close"]
        .to_list()
    )
    return CLOSE_PRICES

TICKER = "WHIRLPOOL.NS"
CLOSE_PRICES = load_close_prices(TICKER)

In [42]:
check_env(StockTradingEnv(CLOSE_PRICES, seed=0))

In [43]:
env = StockTradingEnv(CLOSE_PRICES, seed=0)
obs, info = env.reset()
obs, info

(array([ 0.0000000e+00,  2.4646499e+03,  2.4814500e+03,  2.5205500e+03,
         2.5099500e+03,  2.5145500e+03,  2.5088999e+03,  2.5118501e+03,
         2.4850000e+03,  2.4623000e+03,  2.4597000e+03,  2.4540500e+03,
         2.4569500e+03,  2.4692500e+03,  2.4719500e+03,  2.4890000e+03,
         2.4828501e+03,  2.4750000e+03,  2.4662000e+03,  2.4500000e+03,
         2.4423999e+03,  2.4353999e+03,  2.4407000e+03,  2.4358999e+03,
         2.4340000e+03,  2.4389500e+03,  2.3910000e+03,  2.4107500e+03,
         2.4221001e+03,  2.3888000e+03,  2.3800000e+03,  2.3750000e+03,
         2.3690000e+03,  2.4250000e+03,  2.4750500e+03,  2.4780500e+03,
         2.4925500e+03,  2.5114500e+03,  2.5066499e+03,  2.5090500e+03,
         2.5190000e+03,  9.8977203e+01,  1.0227966e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00, -1.00000

In [44]:
obs[0], obs[1:41]

(0.0,
 array([2464.65, 2481.45, 2520.55, 2509.95, 2514.55, 2508.9 , 2511.85,
        2485.  , 2462.3 , 2459.7 , 2454.05, 2456.95, 2469.25, 2471.95,
        2489.  , 2482.85, 2475.  , 2466.2 , 2450.  , 2442.4 , 2435.4 ,
        2440.7 , 2435.9 , 2434.  , 2438.95, 2391.  , 2410.75, 2422.1 ,
        2388.8 , 2380.  , 2375.  , 2369.  , 2425.  , 2475.05, 2478.05,
        2492.55, 2511.45, 2506.65, 2509.05, 2519.  ], dtype=float32))

In [48]:
obs[-10:]

array([ 0.e+00, -1.e+00,  0.e+00,  1.e+04,  0.e+00,  0.e+00,  0.e+00,
        0.e+00,  1.e+04,  1.e+04], dtype=float32)

In [17]:
from envs.single_stock_trading_past_n_price_portfolio_reward_env import StockTradingEnv

env = StockTradingEnv(CLOSE_PRICES, seed=0)
obs, info = env.reset()
obs, info

(array([  457.65,   458.  ,   448.95,   447.4 ,   446.9 ,   447.4 ,
          447.2 ,   446.9 ,   449.2 ,   460.  ,   459.4 ,   458.55,
          463.45,   462.9 ,   462.8 ,   462.4 ,   465.75,   466.65,
          470.45,   467.75,   464.  ,   460.55,   461.  ,   464.9 ,
          462.95,   465.35,   462.2 ,   462.45,   461.65,   463.4 ,
          461.55,   463.5 ,   458.65,   457.1 ,   458.2 ,   454.8 ,
          454.1 ,   454.  ,   453.4 ,   453.35, 10000.  ,     0.  ,
            0.  ,     0.  ,     0.  , 10000.  , 10000.  ], dtype=float32),
 {})

In [63]:
# {0: "HOLD", 1: "BUY", 2: "SELL"}
s, r, d, t, i = env.step(0)
i["counter"], i["close_price"], i["buy_price"], i["predicted_action"], r

(45, 464.3, 0.0, 'HOLD', 100.0)

In [71]:
470.2 * 22 + 26

10370.4

In [114]:
i

{'seed': 0,
 'counter': 33,
 'close_price': 471.45,
 'predicted_action': 'HOLD',
 'description': '[+] Transaction #33: Profitable Holding. Shares Held: 22.0. Purchase Price per Share: ₹453.35. Current Share Price: ₹471.45. Profit/Loss: ₹398.2001953125 Holding Performance: Good Streak - 33, Bad Streak - 0. Reward Earned: ₹398.2001953125. Portfolio Value: ₹10398.2001953125.Portfolio Value Threshold: ₹10000.0.',
 'available_amount': 26.299805,
 'shares_holdings': 22.0,
 'buy_price': 453.35,
 'buy_price_index': 0,
 'reward': 398.2001953125,
 'done': False,
 'truncated': False,
 'correct_trade': 1,
 'wrong_trade': 0,
 'correct_trade %': 100.0,
 'buy_counter': 1,
 'sell_counter': 0,
 'hold_counter': 33,
 'good_hold_counter': 33,
 'good_sell_counter': 0,
 'good_buy_counter': 0,
 'bad_hold_counter': 0,
 'bad_sell_counter': 0,
 'bad_buy_counter': 1,
 'hold_with_no_shares_counter': 0,
 'bad_holds_with_no_shares_counter': 0,
 'good_holds_with_no_shares_counter': 0,
 'good_hold_streak': 33,
 'bad_

In [53]:
thresh = 20


def simplified_reward(percentage_distance):
    if percentage_distance <= thresh:
        return 100 - (percentage_distance / thresh) * 100
    else:
        return -((percentage_distance - thresh) / thresh) * 100


simplified_reward(15)

25.0